## Δημιουργία μοντέλου λογιστικής παλινδρόμησης - Μάθημα 4

![Γραφικό πληροφοριών: Λογιστική vs. Γραμμική παλινδρόμηση](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Προ-μάθημα κουίζ](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Εισαγωγή

Στο τελευταίο μάθημα για την Παλινδρόμηση, μία από τις βασικές *κλασικές* τεχνικές Μηχανικής Μάθησης, θα εξετάσουμε τη Λογιστική Παλινδρόμηση. Αυτή η τεχνική χρησιμοποιείται για την ανακάλυψη μοτίβων ώστε να προβλέψουμε δυαδικές κατηγορίες. Είναι αυτό το γλυκό σοκολάτα ή όχι; Είναι αυτή η ασθένεια μεταδοτική ή όχι; Θα επιλέξει αυτός ο πελάτης το προϊόν ή όχι;

Σε αυτό το μάθημα, θα μάθετε:

- Τεχνικές για λογιστική παλινδρόμηση

✅ Εμβαθύνετε την κατανόησή σας για την εργασία με αυτόν τον τύπο παλινδρόμησης σε αυτό το [Learn module](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Προαπαιτούμενα

Έχοντας δουλέψει με τα δεδομένα κολοκύθας, είμαστε πλέον αρκετά εξοικειωμένοι ώστε να συνειδητοποιήσουμε ότι υπάρχει μία δυαδική κατηγορία με την οποία μπορούμε να δουλέψουμε: `Color`.

Ας δημιουργήσουμε ένα μοντέλο λογιστικής παλινδρόμησης για να προβλέψουμε, δεδομένων κάποιων μεταβλητών, *ποιο χρώμα είναι πιθανό να έχει μια δεδομένη κολοκύθα* (πορτοκαλί 🎃 ή λευκό 👻).

> Γιατί μιλάμε για δυαδική ταξινόμηση σε ένα μάθημα που αφορά την παλινδρόμηση; Μόνο για γλωσσική ευκολία, καθώς η λογιστική παλινδρόμηση είναι [στην πραγματικότητα μια μέθοδος ταξινόμησης](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), αν και βασίζεται σε γραμμικές αρχές. Μάθετε για άλλους τρόπους ταξινόμησης δεδομένων στην επόμενη ομάδα μαθημάτων.

Για αυτό το μάθημα, θα χρειαστούμε τα παρακάτω πακέτα:

- `tidyverse`: Το [tidyverse](https://www.tidyverse.org/) είναι μια [συλλογή πακέτων R](https://www.tidyverse.org/packages) σχεδιασμένη για να κάνει την επιστήμη δεδομένων πιο γρήγορη, εύκολη και διασκεδαστική!

- `tidymodels`: Το [tidymodels](https://www.tidymodels.org/) είναι ένα [πλαίσιο συλλογής πακέτων](https://www.tidymodels.org/packages/) για μοντελοποίηση και μηχανική μάθηση.

- `janitor`: Το [πακέτο janitor](https://github.com/sfirke/janitor) παρέχει απλά εργαλεία για την εξέταση και τον καθαρισμό "βρώμικων" δεδομένων.

- `ggbeeswarm`: Το [πακέτο ggbeeswarm](https://github.com/eclarke/ggbeeswarm) παρέχει μεθόδους για τη δημιουργία γραφημάτων τύπου beeswarm χρησιμοποιώντας το ggplot2.

Μπορείτε να τα εγκαταστήσετε ως εξής:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Εναλλακτικά, το παρακάτω script ελέγχει αν έχετε τα απαραίτητα πακέτα για την ολοκλήρωση αυτού του μαθήματος και τα εγκαθιστά για εσάς σε περίπτωση που λείπουν.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Ορισμός της ερώτησης**

Για τους σκοπούς μας, θα εκφράσουμε την ερώτηση ως δυαδική: 'Λευκό' ή 'Μη Λευκό'. Υπάρχει επίσης μια κατηγορία 'ριγέ' στο σύνολο δεδομένων μας, αλλά υπάρχουν λίγες περιπτώσεις αυτής, οπότε δεν θα τη χρησιμοποιήσουμε. Εξάλλου, εξαφανίζεται όταν αφαιρούμε τις τιμές null από το σύνολο δεδομένων.

> 🎃 Διασκεδαστικό γεγονός: Μερικές φορές αποκαλούμε τα λευκά κολοκύθια 'κολοκύθια-φαντάσματα'. Δεν είναι πολύ εύκολο να τα σκαλίσουμε, γι' αυτό δεν είναι τόσο δημοφιλή όσο τα πορτοκαλί, αλλά έχουν ενδιαφέρον εμφάνιση! Έτσι, θα μπορούσαμε επίσης να διατυπώσουμε την ερώτησή μας ως: 'Φάντασμα' ή 'Μη Φάντασμα'. 👻

## **Σχετικά με την λογιστική παλινδρόμηση**

Η λογιστική παλινδρόμηση διαφέρει από τη γραμμική παλινδρόμηση, την οποία μάθατε προηγουμένως, σε μερικά σημαντικά σημεία.

#### **Δυαδική ταξινόμηση**

Η λογιστική παλινδρόμηση δεν προσφέρει τις ίδιες δυνατότητες με τη γραμμική παλινδρόμηση. Η πρώτη προσφέρει πρόβλεψη για μια `δυαδική κατηγορία` ("πορτοκαλί ή μη πορτοκαλί"), ενώ η δεύτερη είναι ικανή να προβλέπει `συνεχείς τιμές`, για παράδειγμα, δεδομένης της προέλευσης μιας κολοκύθας και του χρόνου συγκομιδής, *πόσο θα αυξηθεί η τιμή της*.

![Infographic από την Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Άλλες ταξινομήσεις

Υπάρχουν άλλοι τύποι λογιστικής παλινδρόμησης, συμπεριλαμβανομένων των πολυωνυμικών και των διατακτικών:

- **Πολυωνυμική**, που περιλαμβάνει περισσότερες από μία κατηγορίες - "Πορτοκαλί, Λευκό και Ριγέ".

- **Διατακτική**, που περιλαμβάνει διατεταγμένες κατηγορίες, χρήσιμη αν θέλαμε να ταξινομήσουμε τα αποτελέσματά μας λογικά, όπως οι κολοκύθες μας που ταξινομούνται με βάση έναν πεπερασμένο αριθμό μεγεθών (mini, sm, med, lg, xl, xxl).

![Πολυωνυμική vs διατακτική παλινδρόμηση](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Οι μεταβλητές ΔΕΝ χρειάζεται να συσχετίζονται**

Θυμάστε πώς η γραμμική παλινδρόμηση λειτουργούσε καλύτερα με πιο συσχετισμένες μεταβλητές; Η λογιστική παλινδρόμηση είναι το αντίθετο - οι μεταβλητές δεν χρειάζεται να ευθυγραμμίζονται. Αυτό λειτουργεί για αυτά τα δεδομένα που έχουν σχετικά αδύναμες συσχετίσεις.

#### **Χρειάζεστε πολλά καθαρά δεδομένα**

Η λογιστική παλινδρόμηση θα δώσει πιο ακριβή αποτελέσματα αν χρησιμοποιήσετε περισσότερα δεδομένα. Το μικρό μας σύνολο δεδομένων δεν είναι ιδανικό για αυτήν την εργασία, οπότε λάβετε το υπόψη σας.

✅ Σκεφτείτε τους τύπους δεδομένων που θα ήταν κατάλληλοι για λογιστική παλινδρόμηση.

## Άσκηση - καθαρίστε τα δεδομένα

Πρώτα, καθαρίστε λίγο τα δεδομένα, αφαιρώντας τις τιμές null και επιλέγοντας μόνο ορισμένες στήλες:

1. Προσθέστε τον ακόλουθο κώδικα:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Μπορείτε πάντα να ρίξετε μια ματιά στο νέο σας dataframe, χρησιμοποιώντας τη συνάρτηση [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) όπως φαίνεται παρακάτω:


In [ ]:
pumpkins_select %>% 
  glimpse()


Ας επιβεβαιώσουμε ότι θα ασχοληθούμε με ένα πρόβλημα δυαδικής ταξινόμησης:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Οπτικοποίηση - κατηγορικό διάγραμμα
Μέχρι τώρα έχετε φορτώσει ξανά τα δεδομένα για τις κολοκύθες και τα έχετε καθαρίσει ώστε να διατηρήσετε ένα σύνολο δεδομένων που περιλαμβάνει μερικές μεταβλητές, όπως το Χρώμα. Ας οπτικοποιήσουμε το dataframe στο notebook χρησιμοποιώντας τη βιβλιοθήκη ggplot.

Η βιβλιοθήκη ggplot προσφέρει μερικούς έξυπνους τρόπους για να οπτικοποιήσετε τα δεδομένα σας. Για παράδειγμα, μπορείτε να συγκρίνετε τις κατανομές των δεδομένων για κάθε Ποικιλία και Χρώμα σε ένα κατηγορικό διάγραμμα.

1. Δημιουργήστε ένα τέτοιο διάγραμμα χρησιμοποιώντας τη συνάρτηση geombar, με τα δεδομένα για τις κολοκύθες, και καθορίζοντας έναν χρωματικό χάρτη για κάθε κατηγορία κολοκύθας (πορτοκαλί ή λευκή):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Παρατηρώντας τα δεδομένα, μπορείτε να δείτε πώς τα δεδομένα Χρώματος σχετίζονται με την Ποικιλία.

✅ Με βάση αυτό το κατηγοριακό διάγραμμα, ποιες ενδιαφέρουσες εξερευνήσεις μπορείτε να φανταστείτε;


### Προεπεξεργασία δεδομένων: κωδικοποίηση χαρακτηριστικών

Το σύνολο δεδομένων μας για τις κολοκύθες περιέχει τιμές κειμένου για όλες τις στήλες του. Η εργασία με κατηγορικά δεδομένα είναι διαισθητική για τους ανθρώπους, αλλά όχι για τις μηχανές. Οι αλγόριθμοι μηχανικής μάθησης λειτουργούν καλύτερα με αριθμούς. Γι' αυτό η κωδικοποίηση είναι ένα πολύ σημαντικό βήμα στη φάση της προεπεξεργασίας δεδομένων, καθώς μας επιτρέπει να μετατρέψουμε τα κατηγορικά δεδομένα σε αριθμητικά, χωρίς να χάσουμε καμία πληροφορία. Η σωστή κωδικοποίηση οδηγεί στη δημιουργία ενός καλού μοντέλου.

Για την κωδικοποίηση χαρακτηριστικών υπάρχουν δύο κύριοι τύποι κωδικοποιητών:

1. Ordinal encoder: είναι κατάλληλος για τακτικές μεταβλητές, οι οποίες είναι κατηγορικές μεταβλητές όπου τα δεδομένα τους ακολουθούν μια λογική σειρά, όπως η στήλη `item_size` στο σύνολο δεδομένων μας. Δημιουργεί έναν χάρτη έτσι ώστε κάθε κατηγορία να εκπροσωπείται από έναν αριθμό, ο οποίος είναι η σειρά της κατηγορίας στη στήλη.

2. Categorical encoder: είναι κατάλληλος για ονομαστικές μεταβλητές, οι οποίες είναι κατηγορικές μεταβλητές όπου τα δεδομένα τους δεν ακολουθούν μια λογική σειρά, όπως όλα τα χαρακτηριστικά εκτός από το `item_size` στο σύνολο δεδομένων μας. Είναι μια κωδικοποίηση one-hot, που σημαίνει ότι κάθε κατηγορία εκπροσωπείται από μια δυαδική στήλη: η κωδικοποιημένη μεταβλητή είναι ίση με 1 αν η κολοκύθα ανήκει σε αυτήν την ποικιλία και 0 διαφορετικά.

Το Tidymodels παρέχει ένα ακόμη χρήσιμο πακέτο: [recipes](https://recipes.tidymodels.org/) - ένα πακέτο για την προεπεξεργασία δεδομένων. Θα ορίσουμε μια `recipe` που καθορίζει ότι όλες οι στήλες προβλεπτών πρέπει να κωδικοποιηθούν σε ένα σύνολο ακέραιων αριθμών, θα την `prep` για να εκτιμήσουμε τις απαιτούμενες ποσότητες και στατιστικά που χρειάζονται για οποιαδήποτε λειτουργία και τέλος θα την `bake` για να εφαρμόσουμε τους υπολογισμούς σε νέα δεδομένα.

> Συνήθως, το recipes χρησιμοποιείται ως προεπεξεργαστής για τη μοντελοποίηση, όπου καθορίζει ποια βήματα πρέπει να εφαρμοστούν σε ένα σύνολο δεδομένων για να είναι έτοιμο για μοντελοποίηση. Σε αυτήν την περίπτωση είναι **πολύ προτεινόμενο** να χρησιμοποιήσετε ένα `workflow()` αντί να εκτιμάτε χειροκίνητα μια συνταγή χρησιμοποιώντας prep και bake. Θα δούμε όλα αυτά σε λίγο.
>
> Ωστόσο, προς το παρόν, χρησιμοποιούμε recipes + prep + bake για να καθορίσουμε ποια βήματα πρέπει να εφαρμοστούν σε ένα σύνολο δεδομένων για να είναι έτοιμο για ανάλυση δεδομένων και στη συνέχεια να εξάγουμε τα προεπεξεργασμένα δεδομένα με τα βήματα που έχουν εφαρμοστεί.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Ποια είναι τα πλεονεκτήματα της χρήσης ενός ordinal encoder για τη στήλη Item Size;

### Ανάλυση σχέσεων μεταξύ μεταβλητών

Τώρα που έχουμε προεπεξεργαστεί τα δεδομένα μας, μπορούμε να αναλύσουμε τις σχέσεις μεταξύ των χαρακτηριστικών και της ετικέτας, ώστε να αποκτήσουμε μια ιδέα για το πόσο καλά θα μπορέσει το μοντέλο να προβλέψει την ετικέτα με βάση τα χαρακτηριστικά. Ο καλύτερος τρόπος για να πραγματοποιήσουμε αυτό το είδος ανάλυσης είναι η απεικόνιση των δεδομένων. 
Θα χρησιμοποιήσουμε ξανά τη συνάρτηση ggplot geom_boxplot_ για να οπτικοποιήσουμε τις σχέσεις μεταξύ του Item Size, Variety και Color σε μια κατηγορική απεικόνιση. Για καλύτερη απεικόνιση των δεδομένων, θα χρησιμοποιήσουμε την κωδικοποιημένη στήλη Item Size και την μη κωδικοποιημένη στήλη Variety.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Χρησιμοποιήστε ένα swarm plot

Δεδομένου ότι το Color είναι μια δυαδική κατηγορία (Λευκό ή Όχι), απαιτείται 'μια [εξειδικευμένη προσέγγιση](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf) για την οπτικοποίησή του'.

Δοκιμάστε ένα `swarm plot` για να δείξετε την κατανομή του χρώματος σε σχέση με το item_size.

Θα χρησιμοποιήσουμε το [πακέτο ggbeeswarm](https://github.com/eclarke/ggbeeswarm), το οποίο παρέχει μεθόδους για τη δημιουργία διαγραμμάτων τύπου beeswarm χρησιμοποιώντας το ggplot2. Τα διαγράμματα beeswarm είναι ένας τρόπος απεικόνισης σημείων που κανονικά θα επικαλύπτονταν, έτσι ώστε να τοποθετούνται το ένα δίπλα στο άλλο.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Τώρα που έχουμε μια ιδέα για τη σχέση μεταξύ των δυαδικών κατηγοριών του χρώματος και της μεγαλύτερης ομάδας των μεγεθών, ας εξερευνήσουμε τη λογιστική παλινδρόμηση για να καθορίσουμε το πιθανό χρώμα μιας δεδομένης κολοκύθας.

## Δημιουργήστε το μοντέλο σας

Επιλέξτε τις μεταβλητές που θέλετε να χρησιμοποιήσετε στο μοντέλο ταξινόμησης και χωρίστε τα δεδομένα σε σύνολα εκπαίδευσης και δοκιμής. Το [rsample](https://rsample.tidymodels.org/), ένα πακέτο στο Tidymodels, παρέχει υποδομή για αποτελεσματική διαίρεση και επαναδειγματοληψία δεδομένων:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Είμαστε τώρα έτοιμοι να εκπαιδεύσουμε ένα μοντέλο προσαρμόζοντας τα χαρακτηριστικά εκπαίδευσης στην ετικέτα εκπαίδευσης (χρώμα).

Θα ξεκινήσουμε δημιουργώντας μια συνταγή που καθορίζει τα βήματα προεπεξεργασίας που πρέπει να πραγματοποιηθούν στα δεδομένα μας για να τα προετοιμάσουμε για μοντελοποίηση, δηλαδή: κωδικοποίηση κατηγορικών μεταβλητών σε ένα σύνολο ακέραιων αριθμών. Όπως και με το `baked_pumpkins`, δημιουργούμε μια `pumpkins_recipe` αλλά δεν κάνουμε `prep` και `bake`, καθώς αυτά θα ενσωματωθούν σε μια ροή εργασίας, την οποία θα δείτε σε λίγα βήματα από τώρα.

Υπάρχουν αρκετοί τρόποι για να καθορίσετε ένα μοντέλο λογιστικής παλινδρόμησης στο Tidymodels. Δείτε `?logistic_reg()`. Προς το παρόν, θα καθορίσουμε ένα μοντέλο λογιστικής παλινδρόμησης μέσω της προεπιλεγμένης μηχανής `stats::glm()`.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Τώρα που έχουμε μια συνταγή και μια προδιαγραφή μοντέλου, πρέπει να βρούμε έναν τρόπο να τα συνδυάσουμε σε ένα αντικείμενο που θα επεξεργάζεται πρώτα τα δεδομένα (prep+bake στο παρασκήνιο), θα προσαρμόζει το μοντέλο στα επεξεργασμένα δεδομένα και θα επιτρέπει επίσης πιθανές δραστηριότητες μετά την επεξεργασία.

Στο Tidymodels, αυτό το πρακτικό αντικείμενο ονομάζεται [`workflow`](https://workflows.tidymodels.org/) και συγκεντρώνει βολικά τα συστατικά του μοντέλου σας.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Αφού έχει *καθοριστεί* μια ροή εργασίας, ένα μοντέλο μπορεί να `εκπαιδευτεί` χρησιμοποιώντας τη συνάρτηση [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). Η ροή εργασίας θα εκτιμήσει μια συνταγή και θα προεπεξεργαστεί τα δεδομένα πριν από την εκπαίδευση, οπότε δεν θα χρειαστεί να το κάνουμε αυτό χειροκίνητα χρησιμοποιώντας τις prep και bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Το μοντέλο εκτυπώνει τους συντελεστές που έμαθε κατά τη διάρκεια της εκπαίδευσης.

Τώρα που έχουμε εκπαιδεύσει το μοντέλο χρησιμοποιώντας τα δεδομένα εκπαίδευσης, μπορούμε να κάνουμε προβλέψεις στα δεδομένα δοκιμής χρησιμοποιώντας [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Ας ξεκινήσουμε χρησιμοποιώντας το μοντέλο για να προβλέψουμε ετικέτες για το σύνολο δοκιμής μας και τις πιθανότητες για κάθε ετικέτα. Όταν η πιθανότητα είναι μεγαλύτερη από 0.5, η προβλεπόμενη κατηγορία είναι `WHITE`, διαφορετικά είναι `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Πολύ ωραία! Αυτό μας δίνει μια καλύτερη κατανόηση του τρόπου λειτουργίας της λογιστικής παλινδρόμησης.

### Καλύτερη κατανόηση μέσω ενός πίνακα σύγχυσης

Η σύγκριση κάθε πρόβλεψης με την αντίστοιχη "πραγματική" τιμή αναφοράς δεν είναι ένας πολύ αποδοτικός τρόπος για να καθορίσουμε πόσο καλά προβλέπει το μοντέλο. Ευτυχώς, το Tidymodels έχει μερικά ακόμα κόλπα στο μανίκι του: το [`yardstick`](https://yardstick.tidymodels.org/) - ένα πακέτο που χρησιμοποιείται για τη μέτρηση της αποτελεσματικότητας των μοντέλων μέσω μετρικών απόδοσης.

Μία μετρική απόδοσης που σχετίζεται με προβλήματα ταξινόμησης είναι ο [`πίνακας σύγχυσης`](https://wikipedia.org/wiki/Confusion_matrix). Ένας πίνακας σύγχυσης περιγράφει πόσο καλά αποδίδει ένα μοντέλο ταξινόμησης. Καταγράφει πόσα παραδείγματα σε κάθε κατηγορία ταξινομήθηκαν σωστά από το μοντέλο. Στην περίπτωσή μας, θα σας δείξει πόσες πορτοκαλί κολοκύθες ταξινομήθηκαν ως πορτοκαλί και πόσες λευκές κολοκύθες ταξινομήθηκαν ως λευκές· ο πίνακας σύγχυσης δείχνει επίσης πόσες ταξινομήθηκαν σε **λάθος** κατηγορίες.

Η [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) συνάρτηση από το yardstick υπολογίζει αυτή τη διασταυρούμενη ταξινόμηση των παρατηρούμενων και προβλεπόμενων κατηγοριών.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Ας ερμηνεύσουμε τον πίνακα σύγχυσης. Το μοντέλο μας καλείται να ταξινομήσει κολοκύθες μεταξύ δύο δυαδικών κατηγοριών, την κατηγορία `λευκή` και την κατηγορία `μη-λευκή`.

-   Εάν το μοντέλο σας προβλέψει μια κολοκύθα ως λευκή και αυτή ανήκει στην κατηγορία 'λευκή' στην πραγματικότητα, το αποκαλούμε `αληθώς θετικό`, που φαίνεται από τον αριθμό επάνω αριστερά.

-   Εάν το μοντέλο σας προβλέψει μια κολοκύθα ως μη λευκή και αυτή ανήκει στην κατηγορία 'λευκή' στην πραγματικότητα, το αποκαλούμε `ψευδώς αρνητικό`, που φαίνεται από τον αριθμό κάτω αριστερά.

-   Εάν το μοντέλο σας προβλέψει μια κολοκύθα ως λευκή και αυτή ανήκει στην κατηγορία 'μη-λευκή' στην πραγματικότητα, το αποκαλούμε `ψευδώς θετικό`, που φαίνεται από τον αριθμό επάνω δεξιά.

-   Εάν το μοντέλο σας προβλέψει μια κολοκύθα ως μη λευκή και αυτή ανήκει στην κατηγορία 'μη-λευκή' στην πραγματικότητα, το αποκαλούμε `αληθώς αρνητικό`, που φαίνεται από τον αριθμό κάτω δεξιά.

| Αλήθεια |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **Προβλεπόμενο** | ΛΕΥΚΗ | ΠΟΡΤΟΚΑΛΙ |
| ΛΕΥΚΗ        | TP     | FP    |
| ΠΟΡΤΟΚΑΛΙ         | FN     | TN    |

Όπως ίσως έχετε μαντέψει, είναι προτιμότερο να έχουμε μεγαλύτερο αριθμό αληθώς θετικών και αληθώς αρνητικών και μικρότερο αριθμό ψευδώς θετικών και ψευδώς αρνητικών, κάτι που υποδηλώνει ότι το μοντέλο αποδίδει καλύτερα.

Ο πίνακας σύγχυσης είναι χρήσιμος καθώς οδηγεί σε άλλες μετρικές που μπορούν να μας βοηθήσουν να αξιολογήσουμε καλύτερα την απόδοση ενός μοντέλου ταξινόμησης. Ας δούμε μερικές από αυτές:

🎓 Ακρίβεια (Precision): `TP/(TP + FP)` ορίζεται ως το ποσοστό των προβλεπόμενων θετικών που είναι πραγματικά θετικά. Επίσης αποκαλείται [θετική προγνωστική αξία](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Ανάκληση (Recall): `TP/(TP + FN)` ορίζεται ως το ποσοστό των θετικών αποτελεσμάτων από τον αριθμό των δειγμάτων που ήταν πραγματικά θετικά. Επίσης γνωστή ως `ευαισθησία`.

🎓 Ειδικότητα (Specificity): `TN/(TN + FP)` ορίζεται ως το ποσοστό των αρνητικών αποτελεσμάτων από τον αριθμό των δειγμάτων που ήταν πραγματικά αρνητικά.

🎓 Ακρίβεια (Accuracy): `TP + TN/(TP + TN + FP + FN)` Το ποσοστό των ετικετών που προβλέφθηκαν σωστά για ένα δείγμα.

🎓 Μέτρο F (F Measure): Ένας σταθμισμένος μέσος όρος της ακρίβειας και της ανάκλησης, με το καλύτερο να είναι 1 και το χειρότερο να είναι 0.

Ας υπολογίσουμε αυτές τις μετρικές!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Οπτικοποίηση της καμπύλης ROC αυτού του μοντέλου

Ας κάνουμε μία ακόμη οπτικοποίηση για να δούμε την αποκαλούμενη [`καμπύλη ROC`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


Οι καμπύλες ROC χρησιμοποιούνται συχνά για να αποκτήσουμε μια εικόνα της απόδοσης ενός ταξινομητή σε όρους αληθών έναντι ψευδών θετικών. Οι καμπύλες ROC συνήθως απεικονίζουν τον `True Positive Rate`/Ευαισθησία στον άξονα Y και τον `False Positive Rate`/1-Ειδικότητα στον άξονα X. Έτσι, η κλίση της καμπύλης και ο χώρος μεταξύ της γραμμής του μέσου σημείου και της καμπύλης έχουν σημασία: θέλουμε μια καμπύλη που ανεβαίνει γρήγορα και περνάει πάνω από τη γραμμή. Στην περίπτωσή μας, υπάρχουν ψευδώς θετικά στην αρχή, και στη συνέχεια η γραμμή ανεβαίνει και περνάει σωστά πάνω.

Τέλος, ας χρησιμοποιήσουμε το `yardstick::roc_auc()` για να υπολογίσουμε την πραγματική Επιφάνεια Κάτω από την Καμπύλη (Area Under the Curve). Ένας τρόπος ερμηνείας του AUC είναι ως η πιθανότητα το μοντέλο να κατατάξει ένα τυχαίο θετικό παράδειγμα υψηλότερα από ένα τυχαίο αρνητικό παράδειγμα.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Το αποτέλεσμα είναι περίπου `0.975`. Δεδομένου ότι το AUC κυμαίνεται από 0 έως 1, θέλετε ένα υψηλό σκορ, καθώς ένα μοντέλο που είναι 100% σωστό στις προβλέψεις του θα έχει AUC ίσο με 1. Σε αυτή την περίπτωση, το μοντέλο είναι *αρκετά καλό*.

Στα επόμενα μαθήματα για ταξινομήσεις, θα μάθετε πώς να βελτιώσετε τα σκορ του μοντέλου σας (όπως η αντιμετώπιση μη ισορροπημένων δεδομένων σε αυτή την περίπτωση).

## 🚀Πρόκληση

Υπάρχουν πολλά περισσότερα να εξερευνήσετε σχετικά με την λογιστική παλινδρόμηση! Αλλά ο καλύτερος τρόπος για να μάθετε είναι να πειραματιστείτε. Βρείτε ένα σύνολο δεδομένων που ταιριάζει σε αυτόν τον τύπο ανάλυσης και δημιουργήστε ένα μοντέλο με αυτό. Τι μαθαίνετε; συμβουλή: δοκιμάστε το [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) για ενδιαφέροντα σύνολα δεδομένων.

## Ανασκόπηση & Αυτομελέτη

Διαβάστε τις πρώτες σελίδες από [αυτό το άρθρο του Stanford](https://web.stanford.edu/~jurafsky/slp3/5.pdf) σχετικά με κάποιες πρακτικές χρήσεις της λογιστικής παλινδρόμησης. Σκεφτείτε εργασίες που ταιριάζουν καλύτερα σε έναν ή τον άλλο τύπο παλινδρόμησης που έχουμε μελετήσει μέχρι τώρα. Τι θα λειτουργούσε καλύτερα;



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
